### Load the multi-scale map


In [1]:
import sys,os,os.path
os.environ['MODEL_ANNOTATION_ROOT']='Desktop/projects/model_annotation/'

In [2]:
os.getenv("MODEL_ANNOTATION_ROOT")

'Desktop/projects/model_annotation/'

In [3]:
## Parameters to edit
runMode   = "test" # "test",  "full"
sizeThresh = 50 # ToDo: use third quartile value
fixGeneNames = True 
useHGNC_Uniprot = True

### Load the libraries and functions

In [4]:
import os
import pandas as pd
import json
import networkx as nx

from file_io import get_model_directory_path, read_system_json, write_system_json, write_system_tsv, get_root_path
from model_cx2 import get_system, get_genes, getSystemIndex, set_genes
from hugo import get_hugo_data
from uniprot import get_uniprot_data_for_system, summarize_uniprot_features, summarized_uniprot_features_to_tsv
from io import StringIO
from chatgpt_prompts import create_system_prompt_page, create_nesa_chatGPT_prompt, add_uniprot_feature_summary
from pages import write_system_page, write_model_page, dataframe_to_html_table
import cx2_network



/Users/salkhairy/Desktop/projects/cellmaps_annotate_hierarchy/cellmaps_annotate_hierarchy/FixGeneSymbols.py:15: DtypeWarning: Columns (32,34,38,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  hgnc_raw_DF = pd.read_csv('lib/hgnc_idsymbolnamelocus_grouplocus_typestatus.txt', sep = '\t', dtype = {'hgnc_id': str, 'uniprot_ids': str, 'symbol':str})


In [5]:
from FixGeneSymbols import fixGeneSymbol, latestGeneSymbol_2_uniprotID, latestGeneSymbol_2_uniprotID_Dict

In [6]:
def dataframe_to_dict(df):
    """
    Convert a pandas DataFrame into a dictionary indexed by the first column.

    :param df: The pandas DataFrame to convert.
    :return: A dictionary indexed by the first column.
    """
    # Set the index to be the first column
    df = df.set_index(df.columns[0])

    # Convert the DataFrame to a dictionary
    result_dict = df.to_dict(orient='index')

    return result_dict

def make_gene_candidacy_text(gene_data, selected_genes):
    attribute_descriptions = {
        'hasHighConfidenceMut': "Genes with high confidence mutation in ASD-diagnosed individuals:",
        'in_WES_2020': "ASD-risk genes identified in Satterstrom et al., 2020:",
       # 'in_WES_2022': "ASD-risk genes identified in Fu et al., 2022:",
        'connectedToASDPPI': "Proteins connected to ASD-risk proteins (AP-MS experiment):" # ASD-PPI preys

      #  'in_SFARI_cat_2_3': "ASD-risk in SFARI categories 2 and 3:"
    }
    attributes = {key: [] for key in attribute_descriptions.keys()}

    for gene, attributes_data in gene_data.items():
        if gene not in selected_genes:
            continue
        for attribute in attributes.keys():
            if attributes_data[attribute] == 1:
                attributes[attribute].append(gene)

    text_output = ''
    for attribute, genes in attributes.items():
        if len(genes) != 0:
            gene_list = ', '.join(genes)
            text_output += f"{attribute_descriptions[attribute]} {gene_list}\n"

    return text_output.strip()

In [7]:
def create_nesa_system_analysis_page(model_name, version, system_name, protein_list, tsv_data, n_genes=2):


    # Read the TSV data into a DataFrame
    tsv_file = StringIO(tsv_data)
    df = pd.read_csv(tsv_file, sep='\t')

    # Filter the DataFrame based on the n_genes criterion
    df = df[df['Number of Genes'] >= n_genes]
    
    uniprot_table = dataframe_to_html_table(df)

    # Create the ChatGPT analysis section with a placeholder for the analysis text
    chatgpt_analysis = "<h2>ChatGPT 4 Analysis</h2>\n<p>Paste ChatGPT analysis here:</p>\n<!-- Analysis goes here -->"

    page_title = f"{system_name} Summary"
    
    # Create the HTML page with the system summary
    html_page = f"<!DOCTYPE html>\n<html>\n<head>\n<title>{page_title}</title>\n<style>\n \
              body {{background-color: skyblue;}} \n \
              h1, h2 {{color: white; font-family: 'Roboto', sans-serif;}} \n \
              </style>\n</head>\n<body>\n<h1>{system_name} System Summary</h1>\n \
              <h2>{model_name}: {version}</h2>\n \
              \n{chatgpt_analysis}\
              <h2>Proteins</h2>\n<p>{', '.join(protein_list)}</p>\n \
              <h2>UniProt Data</h2>\n{uniprot_table}\n \
              </body>\n</html>"
    
    #html_page = f"<!DOCTYPE html>\n<html>\n<head>\n<title>{page_title}</title>\n</head>\n<body>\n<h1>{system_name} System Summary</h1>\n<h2>Proteins</h2>\n<p>{', '.join(protein_list)}</p>\n<h2>UniProt Data</h2>\n{uniprot_table}\n{chatgpt_analysis}\n</body>\n</html>"

    return html_page

## Multi-scale map level 

In [8]:
model_name = "nesa"
version = "Krogan_230424"
model_cx2_filename = "hidef_50_0.75_5_leiden.edges.cx2"
print(get_model_directory_path(model_name, version))
model_path = os.path.join(get_model_directory_path(model_name, version), model_cx2_filename)
print(model_path)
with open(model_path, encoding='utf-8') as f:
    data = f.read()
    model = json.loads(data)
#print(model)

/Users/salkhairy/Desktop/projects/model_annotation/nesa/Krogan_230424
/Users/salkhairy/Desktop/projects/model_annotation/nesa/Krogan_230424/hidef_50_0.75_5_leiden.edges.cx2


In [9]:
# print(model)

In [10]:
# This NeSA-specific excel spreadsheet contains ASD gene candidacy information
# Set the file path for 'geneCandidacy_DF.xlsx' in the 'nesa' folder
file_path = os.path.join(get_model_directory_path(model_name, version ), 'geneCandidacy_DF.xlsx')  # SA: Note 

# Load the first worksheet of the Excel file into a DataFrame
df = pd.read_excel(file_path, sheet_name=0)

# Convert the DataFrame to a dictionary indexed by the first column
gene_data = dataframe_to_dict(df)


### Perform topological sorting of systems

In [11]:
## Done using R's igraph package because Python has major limitations

In [12]:
import subprocess

In [13]:
modelPath = get_model_directory_path(model_name, version )

In [14]:
edgesFile = 'hidef_50_0.75_5_leiden.edges'

In [ ]:
subprocess.run(['/usr/local/bin/Rscript --vanilla TopologicalSorting.R ' + modelPath + ' ' + edgesFile], shell=True) 


In [ ]:
topologicalSort_DF = pd.read_csv(os.path.join(get_model_directory_path(model_name, version ), 'topologicalSort_DF.txt'), sep="\t", header=None)

In [ ]:
topologicalSort_list = list(topologicalSort_DF.iloc[:,0])

In [ ]:
topologicalSort_list

## System level 

In [ ]:
# system_name_list = ["Cluster5-3", "Cluster4-10", "Cluster1-26", "Cluster3-16" , "Cluster7-0", "Cluster5-8", "Cluster4-14", "Cluster2-41", "Cluster2-20"]

In [15]:
if runMode == "test":
    system_name_list = ["Cluster5-3"]#, "Cluster4-10", "Cluster1-26", "Cluster3-16" , "Cluster7-0", "Cluster5-8", "Cluster4-14", "Cluster2-41", "Cluster2-20"]
else:
    system_name_list = topologicalSort_list[1:5]

In [ ]:
for system_name in system_name_list:
    print("================================================")
    print(system_name)
    ## Select the system and get genes
    system = get_system(model, system_name)
    system["genes_attribute"] = "CD_MemberList"
    genes = get_genes(system)
    
    if len(genes) > sizeThresh:
        break # need to use different approach
        # ToDo: write up approach for larger systems
        
    # print(f'{system_name}: {genes}')
    
    ## Fix names
    if fixGeneNames:
        print("Fixing gene names")
        genes_fixed = [fixGeneSymbol(gene) for gene in genes]
        
       # if testingFixingGenes:
            #genes_fixed = [gene_fixed + '__fixed' for gene_fixed in genes_fixed] # only to make sure using the correct genes downstream
        genes_fixed_str  = ' ' .join(genes_fixed)
        
        ## replace genes with fixed names in the  model itself because every other function is reading from the same name
        set_genes(model, system_name, genes_fixed_str)    
    
    else:
        genes_fixed = genes
        
            
    ## Get the system again because modified names 
    system = get_system(model, system_name)
    
    ## Get HUGO data
    print("Getting HUGO data")
    hugo_data = get_hugo_data(system) 
    
    write_system_json(hugo_data, model_name, version, system_name, "hugo", get_root_path()) 

    ## Get genes from model data for system
    gene_candidacy_text = make_gene_candidacy_text(gene_data, get_genes(system))
    
    ## Get Uniprot Data
    print("Getting Uniprot data")
    
    # SA: here getting Uniprot IDs
   # uniprotIDs = [FixGeneSymbols.latestGeneSymbol_2_uniprotID(gene) for gene in genes_fixed]
    
    # Q: a couple of genes map to multiple uniprot IDs, what to do with them?
    ## Q: How to integrate uniprotIDs with downstream analyses 

    # Q: does the downstream function read_system_json  use the updated model?
    
    # Gathers a protein's function, pathway, disease association, aliases, and summary description data from the uniprot database using its REST api
    hugo_data = read_system_json(model_name, version +'/'+ system_name, system_name, "hugo", get_root_path()) # SA modified
    uniprot_data = get_uniprot_data_for_system(system, useHGNC_Uniprot, hugo_data=hugo_data) # calls FixGeneSymbols.latestGeneSymbol_2_uniprotID
    write_system_json(uniprot_data, model_name, version, system_name, "uniprot", get_root_path()) # SA modified
    
    ## Summarized Features
    # analyze the information to find features shared between n or more system proteins
    print("Summarizing features")
    summarized_features = summarize_uniprot_features(uniprot_data)
    tsv_data = summarized_uniprot_features_to_tsv(summarized_features)
    write_system_tsv(tsv_data, model_name, version +'/' + system_name, system_name, "uniprot_summary", get_root_path()) # SA modified
    tsv_file = StringIO(tsv_data)
    df = pd.read_csv(tsv_file, sep='\t')
    
    ## Create Prompts
    print("Creating prompts")
    prompt = create_nesa_chatGPT_prompt(get_genes(system), tsv_data, gene_candidacy_text =gene_candidacy_text)
    prompt_page = create_system_prompt_page(system_name, prompt)
    write_system_page(prompt_page, model_name, version +'/'+ system_name, system_name, "chatgtp_prompt", get_root_path())
    analysis_page = create_nesa_system_analysis_page(model_name, version, system_name, get_genes(system), tsv_data)
    write_system_page(analysis_page, model_name, version +'/'+ system_name, system_name, "analysis", get_root_path())

    
    ## ToDo: 
        # automatically call chatGPT with prompt
        # save chatGPT response
        # grab the name that chatGPT provided
        # validate references - Ingoo 
    

Cluster5-3
Fixing gene names
Getting HUGO data
Getting Uniprot data
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to map to uniprot ID
using hgnc table to 

In [ ]:
# update the model page to include links to the new pages
write_model_page(model_name, version , get_root_path())

In [ ]:
# return prompt as html or json